## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import re
import glob
%matplotlib inline

nx = 9
ny = 6

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
cal_img_dir = './camera_cal/'

cal_images = []
for fname in glob.glob(cal_img_dir+"*.jpg"):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (nx,ny), corners,ret)
        cal_images.append(img)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, cal_images[0].shape[::-1][1:], None, None)

## Helper Functions

In [2]:
def warp(img, src_pts, dest_pts):
    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])

    # For source points I'm grabbing the outer four detected corners
    src = src_pts
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = dest_pts
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)

    # Return the resulting image and matrix
    return warped, M

## Apply Gradient & Color Thresholding

In [3]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    graddir = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(graddir)
    binary_output[(graddir >= thresh[0]) & (graddir <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

In [4]:
def grad_mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # Return the binary image
    return binary_output

In [5]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    sobel = None
    if orient == "x":
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        sobel = sobelx
    else:
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        sobel = sobely
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.abs(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # 6) Return this mask as your binary_output image
    binary_output = np.copy(sxbinary) # Remove this line
    return binary_output

In [6]:
# Edit this function to create your own pipeline.
def grad_col_thresholding(
    img
):
    # Grayscale image
    g_thresh = (180, 255)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    g_binary = np.zeros_like(gray)
    g_binary[(gray > g_thresh[0]) & (gray <= g_thresh[1])] = 1
    
    # Grad Magnitude
    mag_thresh=(30, 100)
    mag_binary = grad_mag_thresh(img, 3, mag_thresh)
    
    # sobel x
    sx_thresh = (20, 100)
    sx_binary = abs_sobel_thresh(img, orient='x', thresh_min=sx_thresh[0], thresh_max=sx_thresh[1])
    
    # sobel y
    sy_thresh = (20, 100)
    sy_binary = abs_sobel_thresh(img, orient='y', thresh_min=sy_thresh[0], thresh_max=sy_thresh[1])
    
    # Grad Direction
    dir_binary = dir_threshold(img, sobel_kernel=15, thresh=(0.7, 1.3))
    
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hsv[:,:,0]
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # Grab R channel
    rgb = img
    r_channel = rgb[:,:,0]
    
    # Threshold color channels
    s_thresh=(170, 255)
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    h_thresh=(15, 100)
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    r_thresh=(200, 255)
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1
    
    # Stack each channel
    combined = np.zeros_like(s_binary)
    combined[
        (((sx_thresh == 1) & (sy_thresh == 1)) | ((mag_binary == 1) & (dir_binary == 1))) |
        (
            (g_binary == 1) &
            (r_binary == 1) &
            (s_binary == 1) &
            (h_binary == 1) 
        )
    ] = 1
    return combined

## Measurement of Curvature

In [7]:
def curve_fit_measurement(leftx, lefty, rightx, righty, ploty):
    # Fit a second order polynomial to each lane line
    left_fit = np.polyfit(lefty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(righty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # measure curvature in radians
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Now our radius of curvature is in meters
    curverad = (left_curverad + right_curverad) / 2
    
    # Find the position of the car
    lane_pos = np.mean(np.concatenate([left_fitx, right_fitx], axis=0)) * xm_per_pix
    
    return (left_fitx, right_fitx, curverad, lane_pos)

## Locate Lane Lines and Fit a Polynomial

In [8]:
def curve_lane_detect(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # return lane curves & curvature measurements
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx, right_fitx, curverad, lane_pos = curve_fit_measurement(
        leftx, lefty, rightx, righty, ploty
    )
    curve_measurement = curverad
    
    # Find Car Position
    car_pos = (binary_warped.shape[1] / 2) * xm_per_pix - lane_pos
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    left_line_pts = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    line_pts = np.hstack((left_line_pts, right_line_pts))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([line_pts]), (0,255, 0))
    result_image = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    return (result_image, curve_measurement, car_pos)

## Main Pipeline for Detecting Lane Lines

In [9]:
def pipeline(image):
    # undistort the image
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    
    # threshold on the image
    threshold_image = grad_col_thresholding(undist)

    # perspective transform the image
    img = threshold_image
    img_size = [img.shape[1], img.shape[0]]
    offset = 100
    src_pts = np.float32(
        [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
        [((img_size[0] / 6) - 10), img_size[1]],
        [(img_size[0] * 5 / 6) + 60, img_size[1]],
        [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
    dest_pts = np.float32(
        [[(img_size[0] / 4), 0],
        [(img_size[0] / 4), img_size[1]],
        [(img_size[0] * 3 / 4), img_size[1]],
        [(img_size[0] * 3 / 4), 0]])
    p_transformed, M = warp(img, src_pts, dest_pts)

    # detect lane lines & curves & car position
    p_transformed_curve, curve_measurement, car_pos = curve_lane_detect(p_transformed)

    # warp the output back to original perspective
    warped, M2 = warp(p_transformed_curve, dest_pts, src_pts)
    warpage = np.array(warped,np.uint8)
    # overlay image with lane lines
    output = undist
    cv2.putText(output, "curvature radius: "+"{0:.2f}".format(curve_measurement)+"m",(10,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
    if car_pos > 0:
        
        cv2.putText(output, "vehicle position: "+"{0:.2f}".format(car_pos)+"m right of center",(10,300), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
    else:
        cv2.putText(output, "vehicle position: "+"{0:.2f}".format(-car_pos)+"m left of center",(10,300), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
    cv2.addWeighted(warpage, 0.5, output, 1, 0, output)
    return output

## Go Through Every Stage of My Pipeline

In [10]:
# read in an image
test_image_paths = glob.glob("./test_images/*.jpg")
test_image = mpimg.imread(test_image_paths[0])
plt.imsave("./output_images/orignal.jpg", test_image)

In [11]:
# undistort the image
undist = cv2.undistort(test_image, mtx, dist, None, mtx)
plt.imsave("./output_images/undistorted.jpg", undist)

In [12]:
# threshold on the image
threshold_image = grad_col_thresholding(undist)
plt.imsave("./output_images/threshold.jpg", threshold_image)

In [13]:
# perspective transform the image
img = threshold_image
img_size = [img.shape[1], img.shape[0]]
offset = 100
src_pts = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
dest_pts = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])
p_transformed, M = warp(img, src_pts, dest_pts)
plt.imsave("./output_images/perspective_transformed.jpg", p_transformed)

In [14]:
# detect lane lines & curves
p_transformed_lane, curve_measurement, car_pos = curve_lane_detect(p_transformed)
plt.imsave("./output_images/lane.jpg", p_transformed_lane)

In [15]:
# warp the output back to original perspective
warped, M2 = warp(p_transformed_lane, dest_pts, src_pts)
warpage = np.array(warped,np.uint8)
plt.imsave("./output_images/warped_back.jpg", warpage)

In [16]:
# overlay image with lane lines
output = undist
cv2.putText(output, "curvature radius: "+"{0:.2f}".format(curve_measurement)+"m",(10,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
if car_pos > 0:
    cv2.putText(output, "vehicle position: "+"{0:.2f}".format(car_pos)+"m right of center",(10,300), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
else:
    cv2.putText(output, "vehicle position: "+"{0:.2f}".format(-car_pos)+"m left of center",(10,300), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2,cv2.LINE_AA)
cv2.addWeighted(warpage, 0.5, output, 1, 0, output)
plt.imsave("./output_images/final.jpg", output)

## Run Pipeline through video file

In [17]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [18]:
clip_output = 'project_output_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(clip_output, audio=False)

[MoviePy] >>>> Building video project_output_video.mp4
[MoviePy] Writing video project_output_video.mp4


100%|█████████▉| 1260/1261 [15:09<00:00,  1.14it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output_video.mp4 

CPU times: user 11min 52s, sys: 4min 40s, total: 16min 32s
Wall time: 15min 10s
